In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [126]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [127]:
obs = pd.read_csv("../data/obs_windows/observation_7D_prediction_24H_48H_72H.csv")

C:\Users\Conrad\AppData\Local\Temp\ipykernel_21292\3120826012.py:1: DtypeWarning: Columns (3,4,7,8,10,11,12,13,14,15,16,17,19,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv("../data/obs_windows/observation_7D_prediction_24H_48H_72H.csv")


### 7d observation window with 24hr prediction window

In [128]:
obs_24 = obs.drop(obs.loc[:, 'spn':'full_derate_within_24hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_48hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [129]:
eq_ids = obs_24.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [130]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [131]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [132]:
#merge modeling column onto original dataframe
obs_24 = obs_24.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_24 = obs_24.merge(eq_ids, on='EquipmentID')
obs_24 = obs_24.drop(columns='EquipmentID')

In [133]:
#creating training, testing, and validation dfs
obs_24_training = obs_24[obs_24['modeling']=='training']
obs_24_training = obs_24_training.drop(columns=['modeling', 'random'])
print(obs_24_training.shape)

obs_24_test = obs_24[obs_24['modeling']=='testing']
obs_24_test = obs_24_test.drop(columns=['modeling', 'random'])
print(obs_24_test.shape)

obs_24_validation = obs_24[obs_24['modeling']=='validation']
obs_24_validation = obs_24_validation.drop(columns=['modeling', 'random'])
print(obs_24_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [134]:
df_sampling = obs_24_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_24hr'] == 0][0:18855]

In [135]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_24hr']==1], df_sampling_non_tenth])

In [136]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_24hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [137]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [138]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_24hr']==1]
smote_derate.shape

(18855, 56)

In [139]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_24hr']==0]])
und_samp.shape

(215061, 56)

In [140]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_24hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_24_test[features]
y_test = obs_24_test[target]

In [141]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [142]:
print(y_resampled.shape)

(56565, 1)


In [143]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [144]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [145]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,3.254962e-01
0,Aftertreatment System,2.450280e-01
51,Unknown,1.333503e-01
18,Engine Coolant,7.171544e-02
22,Engine Fuel,3.427875e-02
28,Engine Protection Torque Full Derate,2.581604e-02
43,J1939 Network,2.395421e-02
52,Variable Geometry Turbocharger,2.289851e-02
41,Intake Manifold,2.130903e-02
39,Injector Metering Rail,1.704484e-02


In [146]:
confusion_matrix(y_test, y_pred)

array([[190317,   7695],
       [  1266,   3471]], dtype=int64)

In [147]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [148]:
(tn, fp, fn, tp)

(190317, 7695, 1266, 3471)

In [149]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98    198012
         1.0       0.31      0.73      0.44      4737

    accuracy                           0.96    202749
   macro avg       0.65      0.85      0.71    202749
weighted avg       0.98      0.96      0.96    202749



In [150]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [151]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9211188136613722


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 7d observation window with 48hr prediction window

In [152]:
obs_48 = obs.drop(obs.loc[:, 'spn':'full_derate_within_48hr'], axis=1).drop(obs.loc[:, 'partial_derate_within_72hr':'derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [153]:
eq_ids = obs_48.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [154]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [155]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [156]:
#merge modeling column onto original dataframe
obs_48 = obs_48.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_48 = obs_48.merge(eq_ids, on='EquipmentID')
obs_48 = obs_48.drop(columns='EquipmentID')

In [157]:
#creating training, testing, and validation dfs
obs_48_training = obs_48[obs_48['modeling']=='training']
obs_48_training = obs_48_training.drop(columns=['modeling', 'random'])
print(obs_48_training.shape)

obs_48_test = obs_48[obs_48['modeling']=='testing']
obs_48_test = obs_48_test.drop(columns=['modeling', 'random'])
print(obs_48_test.shape)

obs_48_validation = obs_48[obs_48['modeling']=='validation']
obs_48_validation = obs_48_validation.drop(columns=['modeling', 'random'])
print(obs_48_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [158]:
df_sampling = obs_48_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_48hr'] == 0][0:18855]

In [159]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_48hr']==1], df_sampling_non_tenth])

In [160]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_48hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

X_test = obs_48_test[features]
y_test = obs_48_test[target]

In [161]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [162]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_48hr']==1]
smote_derate.shape

(18855, 56)

In [163]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_48hr']==0]])
und_samp.shape

(214184, 56)

In [164]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_48hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

In [165]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [166]:
print(y_resampled.shape)

(56565, 1)


In [167]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [168]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [169]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,2.997240e-01
0,Aftertreatment System,2.335250e-01
51,Unknown,1.476410e-01
18,Engine Coolant,7.861727e-02
22,Engine Fuel,4.208726e-02
43,J1939 Network,3.049233e-02
28,Engine Protection Torque Full Derate,2.279377e-02
52,Variable Geometry Turbocharger,2.176359e-02
41,Intake Manifold,1.860147e-02
39,Injector Metering Rail,1.752685e-02


In [170]:
confusion_matrix(y_test, y_pred)

array([[189463,   7865],
       [  1711,   3710]], dtype=int64)

In [171]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [172]:
(tn, fp, fn, tp)

(189463, 7865, 1711, 3710)

In [173]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98    197328
         1.0       0.32      0.68      0.44      5421

    accuracy                           0.95    202749
   macro avg       0.66      0.82      0.71    202749
weighted avg       0.97      0.95      0.96    202749



In [174]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [175]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8932536057675948


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 7d observation window with 72hr prediction window

In [176]:
obs_72 = obs.drop(obs.loc[:, 'spn':'full_derate_within_72hr'], axis=1).drop(columns='EventTimeStamp')

In [177]:
eq_ids = obs_72.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1039

In [178]:
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

In [179]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [180]:
#merge modeling column onto original dataframe
obs_72 = obs_72.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

obs_72 = obs_72.merge(eq_ids, on='EquipmentID')
obs_72 = obs_72.drop(columns='EquipmentID')

In [181]:
#creating training, testing, and validation dfs
obs_72_training = obs_72[obs_72['modeling']=='training']
obs_72_training = obs_72_training.drop(columns=['modeling', 'random'])
print(obs_72_training.shape)

obs_72_test = obs_72[obs_72['modeling']=='testing']
obs_72_test = obs_72_test.drop(columns=['modeling', 'random'])
print(obs_72_test.shape)

obs_72_validation = obs_72[obs_72['modeling']=='validation']
obs_72_validation = obs_72_validation.drop(columns=['modeling', 'random'])
print(obs_72_validation.shape)

(200860, 56)
(202749, 56)
(151246, 56)


In [182]:
df_sampling = obs_72_training
df_sampling_non_tenth = df_sampling[df_sampling['derate_within_72hr'] == 0][0:18855]

In [183]:
df_sampling_derate_tenth = pd.concat([df_sampling[df_sampling['derate_within_72hr']==1], df_sampling_non_tenth])

In [184]:
oversampler = SMOTE(k_neighbors=5, random_state=321)

features = df_sampling_derate_tenth.loc[:, 'Aftertreatment System':].columns.tolist()

target = ['derate_within_72hr']

X_train = df_sampling_derate_tenth[features]
y_train = df_sampling_derate_tenth[target]

In [185]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [186]:
smote_all = pd.concat([y_smote, X_smote], axis=1)
smote_derate = smote_all[smote_all['derate_within_72hr']==1]
smote_derate.shape

(18855, 56)

In [187]:
und_samp = pd.concat([smote_derate, df_sampling[df_sampling['derate_within_72hr']==0]])
und_samp.shape

(213477, 56)

In [188]:
undersampler = RandomUnderSampler(random_state=321, sampling_strategy=1/2)

features = und_samp.loc[:, 'Aftertreatment System':].columns.tolist()

target =['derate_within_72hr']

X_train_final = und_samp[features]
y_train_final = und_samp[target]

X_test = obs_72_test[features]
y_test = obs_72_test[target]

In [189]:
X_resampled, y_resampled = undersampler.fit_resample(X_train_final, y_train_final)

In [190]:
print(y_resampled.shape)

(56565, 1)


In [191]:
rf_pipeline = Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('rf_clf', RandomForestClassifier(n_estimators = 500, n_jobs = -1, random_state = 69))
    ]
)

In [192]:
rand_forest = rf_pipeline.fit(X_resampled, y_resampled)
rf_pipeline.score(X_test, y_test)

y_pred = rand_forest.predict(X_test)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


In [193]:
data = {'feature': np.array(X_resampled.columns), 'importance': rf_pipeline.named_steps['rf_clf'].feature_importances_}
pd.DataFrame(data).sort_values('importance', ascending = False)


,feature,importance
29,Engine Protection Torque Partial Derate,0.276446
0,Aftertreatment System,0.228875
51,Unknown,0.154881
18,Engine Coolant,0.083103
22,Engine Fuel,0.046791
43,J1939 Network,0.035276
28,Engine Protection Torque Full Derate,0.023745
52,Variable Geometry Turbocharger,0.022511
41,Intake Manifold,0.020049
17,Engine Control Module,0.017226


In [194]:
confusion_matrix(y_test, y_pred)

array([[188627,   8224],
       [  2086,   3812]], dtype=int64)

In [195]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [196]:
(tn, fp, fn, tp)

(188627, 8224, 2086, 3812)

In [197]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.97    196851
         1.0       0.32      0.65      0.43      5898

    accuracy                           0.95    202749
   macro avg       0.65      0.80      0.70    202749
weighted avg       0.97      0.95      0.96    202749



In [198]:
lg_pipeline =  Pipeline(
    steps = [
        ('scale', MinMaxScaler()),
        ('lg_clf', LogisticRegression())
    ]
)

In [199]:
logistic = lg_pipeline.fit(X_resampled, y_resampled)

y_pred2 = logistic.predict(X_test)

y_pred_proba = logistic.predict_proba(X_test)[::,1]

auc = metrics.roc_auc_score(y_test, y_pred_proba)

print(auc)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8756279700004066


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
